# House Price Prediction - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on the house price dataset to understand patterns, relationships, and data quality issues that will inform our modeling approach.

## Objectives
1. Load and validate the dataset
2. Analyze data quality and missing values
3. Explore target variable distribution
4. Investigate feature relationships and correlations
5. Identify patterns and insights for feature engineering
6. Generate actionable recommendations for data preprocessing

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Custom modules
import sys
sys.path.append('..')

from src.data_loader import load_data_with_fallback, validate_dataset
from src.eda import EDAAnalyzer, compare_train_test_distributions
from src.data_quality import DataQualityAssessor
from src.utils import print_data_info

# Configure display and plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

print("📊 EDA Environment Setup Complete")
print("📁 Loading house price data...")

In [ ]:
# Load data with fallback to sample data if Kaggle dataset not available
train_data, test_data = load_data_with_fallback()

# Validate dataset structure
is_valid = validate_dataset(train_data, test_data)

if is_valid:
    print("\n✅ Dataset validation successful!")
    print(f"📈 Training data shape: {train_data.shape}")
    print(f"📊 Test data shape: {test_data.shape}")
    print(f"🔍 Total features: {train_data.shape[1] - 1}")
else:
    print("❌ Dataset validation failed!")

## 2. Data Quality Assessment

In [ ]:
# Initialize data quality assessor
quality_assessor = DataQualityAssessor(train_data, "Training Data")

# Generate comprehensive quality report
quality_report = quality_assessor.generate_quality_report()

# Print quality summary
quality_assessor.print_summary()

In [ ]:
# Detailed missing values analysis
missing_analysis = quality_report['missing_values']

print("📊 MISSING VALUES DETAILED ANALYSIS")
print("=" * 50)

print(f"\nSummary:")
print(f"  • Total missing values: {missing_analysis['summary']['total_missing_values']:,}")
print(f"  • Overall missing percentage: {missing_analysis['summary']['missing_percentage_overall']:.2f}%")
print(f"  • Columns with missing data: {missing_analysis['summary']['columns_with_missing']}")
print(f"  • Complete columns: {missing_analysis['summary']['complete_columns']}")

print(f"\nBy Severity:")
for severity, columns in missing_analysis['by_severity'].items():
    if columns:
        print(f"  • {severity.replace('_', ' ').title()}: {len(columns)} columns")
        if len(columns) <= 5:
            print(f"    {columns}")
        else:
            print(f"    {columns[:5]}... (and {len(columns)-5} more)")

## 3. Target Variable Analysis

In [ ]:
# Initialize EDA analyzer
eda_analyzer = EDAAnalyzer(train_data, test_data)

# Analyze target variable
target_analysis = eda_analyzer.analyze_target_variable()

print("🎯 TARGET VARIABLE ANALYSIS (SalePrice)")
print("=" * 50)

if 'error' not in target_analysis:
    print(f"\nDescriptive Statistics:")
    print(f"  • Count: {target_analysis['count']:,}")
    print(f"  • Mean: ${target_analysis['mean']:,.0f}")
    print(f"  • Median: ${target_analysis['median']:,.0f}")
    print(f"  • Standard Deviation: ${target_analysis['std']:,.0f}")
    print(f"  • Range: ${target_analysis['min']:,.0f} - ${target_analysis['max']:,.0f}")
    print(f"  • Skewness: {target_analysis['skewness']:.3f}")
    print(f"  • Kurtosis: {target_analysis['kurtosis']:.3f}")
    
    print(f"\nPrice Distribution:")
    ranges = target_analysis['price_ranges']
    total = sum(ranges.values())
    for range_name, count in ranges.items():
        percentage = count / total * 100
        print(f"  • {range_name.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")
    
    print(f"\nOutlier Analysis:")
    outliers = target_analysis['outliers']
    print(f"  • Outlier count: {outliers['outlier_count']} ({outliers['outlier_percentage']:.1f}%)")
    print(f"  • Outlier bounds: ${outliers['lower_bound']:,.0f} - ${outliers['upper_bound']:,.0f}")
else:
    print(f"❌ Error in target analysis: {target_analysis['error']}")

In [ ]:
# Visualize target variable distribution
eda_analyzer.plot_target_distribution(figsize=(18, 6))

## 4. Missing Values Visualization

In [ ]:
# Plot missing values
eda_analyzer.plot_missing_values(figsize=(15, 10), top_n=20)

In [ ]:
# Missing value patterns analysis
missing_patterns = quality_report['missing_values']['patterns']

print("🔍 MISSING VALUE PATTERNS")
print("=" * 50)

print(f"\nRow-level Analysis:")
print(f"  • Rows with missing data: {missing_patterns['rows_with_missing']:,}")
print(f"  • Complete rows: {missing_patterns['rows_complete']:,}")
print(f"  • Complete rows percentage: {missing_patterns['complete_rows_percentage']:.1f}%")

if missing_patterns['highly_correlated_missing']:
    print(f"\nHighly Correlated Missing Values:")
    for corr_pair in missing_patterns['highly_correlated_missing'][:5]:
        print(f"  • {corr_pair['feature_1']} ↔ {corr_pair['feature_2']}: {corr_pair['correlation']:.3f}")
else:
    print(f"\n✅ No highly correlated missing value patterns found")

## 5. Feature Correlation Analysis

In [ ]:
# Analyze correlations with target variable
if 'SalePrice' in train_data.columns:
    top_correlations = eda_analyzer.target_correlation(top_n=15)
    
    print("🔗 TOP FEATURES CORRELATED WITH SALEPRICE")
    print("=" * 50)
    
    for idx, row in top_correlations.iterrows():
        direction = "📈" if row['correlation'] > 0 else "📉"
        print(f"{idx+1:2d}. {direction} {row['feature']:<20} : {row['correlation']:>7.3f}")
else:
    print("⚠️ SalePrice column not found for correlation analysis")

In [ ]:
# Plot correlation heatmap for top features
if 'SalePrice' in train_data.columns:
    eda_analyzer.plot_correlation_heatmap(figsize=(14, 12), top_n=15)
else:
    print("⚠️ Cannot plot correlation heatmap without SalePrice column")

## 6. Numerical Features Analysis

In [ ]:
# Analyze numerical features
numerical_stats = eda_analyzer.analyze_numerical_features()

print("🔢 NUMERICAL FEATURES ANALYSIS")
print("=" * 50)

print(f"\nTotal numerical features: {len(numerical_stats)}")

# Features with high skewness
high_skew_features = [(name, stats['skewness']) for name, stats in numerical_stats.items() 
                      if abs(stats['skewness']) > 1]
high_skew_features.sort(key=lambda x: abs(x[1]), reverse=True)

if high_skew_features:
    print(f"\nHighly Skewed Features (|skewness| > 1):")
    for feature, skewness in high_skew_features[:10]:
        direction = "right" if skewness > 0 else "left"
        print(f"  • {feature:<20} : {skewness:>7.2f} ({direction} skewed)")

# Features with many zeros
high_zero_features = [(name, stats['zeros_count'], stats['zeros_count']/len(train_data)*100) 
                      for name, stats in numerical_stats.items() 
                      if stats['zeros_count']/len(train_data) > 0.3]
high_zero_features.sort(key=lambda x: x[2], reverse=True)

if high_zero_features:
    print(f"\nFeatures with Many Zeros (>30%):")
    for feature, count, percentage in high_zero_features[:10]:
        print(f"  • {feature:<20} : {count:>5} ({percentage:>5.1f}%)")

In [ ]:
# Plot distributions of key numerical features
key_numerical_features = ['GrLivArea', 'TotalBsmtSF', 'LotArea', 'YearBuilt', 'OverallQual']
existing_features = [f for f in key_numerical_features if f in train_data.columns]

if existing_features:
    n_features = len(existing_features)
    n_cols = 3
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, feature in enumerate(existing_features):
        row = i // n_cols
        col = i % n_cols
        
        ax = axes[row, col]
        
        # Plot histogram
        train_data[feature].hist(bins=30, ax=ax, alpha=0.7, color='skyblue', edgecolor='black')
        ax.set_title(f'{feature} Distribution')
        ax.set_xlabel(feature)
        ax.set_ylabel('Frequency')
        
        # Add statistics
        if feature in numerical_stats:
            stats = numerical_stats[feature]
            ax.axvline(stats['mean'], color='red', linestyle='--', alpha=0.7, label=f"Mean: {stats['mean']:.0f}")
            ax.axvline(stats['median'], color='green', linestyle='--', alpha=0.7, label=f"Median: {stats['median']:.0f}")
            ax.legend()
    
    # Hide empty subplots
    for i in range(len(existing_features), n_rows * n_cols):
        row = i // n_cols
        col = i % n_cols
        axes[row, col].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Key numerical features not found in dataset")

## 7. Categorical Features Analysis

In [ ]:
# Analyze categorical features
categorical_stats = eda_analyzer.analyze_categorical_features()

print("📝 CATEGORICAL FEATURES ANALYSIS")
print("=" * 50)

print(f"\nTotal categorical features: {len(categorical_stats)}")

# High cardinality features
high_cardinality = [(name, stats['unique_values']) for name, stats in categorical_stats.items() 
                    if stats['unique_values'] > 20]
high_cardinality.sort(key=lambda x: x[1], reverse=True)

if high_cardinality:
    print(f"\nHigh Cardinality Features (>20 unique values):")
    for feature, unique_count in high_cardinality:
        print(f"  • {feature:<20} : {unique_count:>3} unique values")

# Low cardinality features
low_cardinality = [(name, stats['unique_values']) for name, stats in categorical_stats.items() 
                   if stats['unique_values'] <= 5]
low_cardinality.sort(key=lambda x: x[1])

if low_cardinality:
    print(f"\nLow Cardinality Features (≤5 unique values):")
    for feature, unique_count in low_cardinality:
        print(f"  • {feature:<20} : {unique_count:>3} unique values")

# Features with missing values
categorical_missing = [(name, stats['missing_count'], stats['missing_count']/len(train_data)*100) 
                       for name, stats in categorical_stats.items() 
                       if stats['missing_count'] > 0]
categorical_missing.sort(key=lambda x: x[2], reverse=True)

if categorical_missing:
    print(f"\nCategorical Features with Missing Values:")
    for feature, count, percentage in categorical_missing[:10]:
        print(f"  • {feature:<20} : {count:>5} ({percentage:>5.1f}%)")

In [ ]:
# Visualize relationship between key categorical features and target
key_categorical_features = ['Neighborhood', 'OverallQual', 'ExterQual', 'KitchenQual']
existing_cat_features = [f for f in key_categorical_features if f in train_data.columns]

if existing_cat_features and 'SalePrice' in train_data.columns:
    n_features = min(4, len(existing_cat_features))  # Limit to 4 for better visualization
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    axes = axes.flatten()
    
    for i, feature in enumerate(existing_cat_features[:n_features]):
        ax = axes[i]
        
        # Create boxplot
        sns.boxplot(data=train_data, x=feature, y='SalePrice', ax=ax)
        ax.set_title(f'SalePrice by {feature}', fontsize=14)
        ax.tick_params(axis='x', rotation=45)
        
        # Format y-axis to show prices in thousands
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
    
    # Hide empty subplots
    for i in range(n_features, 4):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
else:
    if 'SalePrice' not in train_data.columns:
        print("⚠️ SalePrice column not found for categorical analysis")
    else:
        print("⚠️ Key categorical features not found in dataset")

## 8. Feature Relationships and Scatter Plots

In [ ]:
# Create scatter plots for key feature relationships
if 'SalePrice' in train_data.columns:
    feature_pairs = [
        ('GrLivArea', 'SalePrice'),
        ('TotalBsmtSF', 'SalePrice'),
        ('YearBuilt', 'SalePrice'),
        ('OverallQual', 'SalePrice')
    ]
    
    # Filter to existing features
    existing_pairs = [(f1, f2) for f1, f2 in feature_pairs 
                      if f1 in train_data.columns and f2 in train_data.columns]
    
    if existing_pairs:
        n_pairs = len(existing_pairs)
        n_cols = 2
        n_rows = (n_pairs + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6*n_rows))
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for i, (feature1, feature2) in enumerate(existing_pairs):
            row = i // n_cols
            col = i % n_cols
            ax = axes[row, col]
            
            # Create scatter plot
            ax.scatter(train_data[feature1], train_data[feature2], alpha=0.6, s=20)
            ax.set_xlabel(feature1)
            ax.set_ylabel(feature2)
            ax.set_title(f'{feature1} vs {feature2}')
            
            # Add correlation coefficient
            if train_data[feature1].dtype in ['int64', 'float64'] and train_data[feature2].dtype in ['int64', 'float64']:
                corr = train_data[feature1].corr(train_data[feature2])
                ax.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
                       transform=ax.transAxes, 
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Hide empty subplots
        for i in range(len(existing_pairs), n_rows * n_cols):
            row = i // n_cols
            col = i % n_cols
            axes[row, col].set_visible(False)
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No suitable feature pairs found for scatter plots")
else:
    print("⚠️ SalePrice column not found for relationship analysis")

## 9. Data Distribution Analysis

In [ ]:
# Analyze data distributions
distribution_analysis = eda_analyzer.data_distribution_analysis()

print("📊 DATA DISTRIBUTION ANALYSIS")
print("=" * 50)

# Group features by distribution type
distribution_groups = {}
for feature, analysis in distribution_analysis.items():
    dist_type = analysis['distribution_type']
    if dist_type not in distribution_groups:
        distribution_groups[dist_type] = []
    distribution_groups[dist_type].append((feature, analysis['skewness']))

for dist_type, features in distribution_groups.items():
    print(f"\n{dist_type.replace('_', ' ').title()} ({len(features)} features):")
    for feature, skewness in sorted(features, key=lambda x: abs(x[1]), reverse=True)[:5]:
        print(f"  • {feature:<20} (skewness: {skewness:>6.2f})")
    if len(features) > 5:
        print(f"    ... and {len(features) - 5} more")

## 10. Outlier Analysis

In [ ]:
# Analyze outliers in key numerical features
outlier_analysis = quality_assessor.assess_outliers()

print("🎯 OUTLIER ANALYSIS")
print("=" * 50)

# Features with high outlier percentages
high_outlier_features = []
for feature, analysis in outlier_analysis.items():
    outlier_pct = analysis['iqr_method']['outlier_percentage']
    if outlier_pct > 5:  # More than 5% outliers
        high_outlier_features.append((feature, outlier_pct, analysis['iqr_method']['outlier_count']))

high_outlier_features.sort(key=lambda x: x[1], reverse=True)

if high_outlier_features:
    print(f"\nFeatures with High Outlier Rates (>5%):")
    for feature, pct, count in high_outlier_features[:10]:
        print(f"  • {feature:<20} : {count:>4} outliers ({pct:>5.1f}%)")
else:
    print("\n✅ No features with high outlier rates detected")

# Summary statistics
total_outliers = sum(analysis['iqr_method']['outlier_count'] for analysis in outlier_analysis.values())
avg_outlier_pct = np.mean([analysis['iqr_method']['outlier_percentage'] for analysis in outlier_analysis.values()])

print(f"\nOutlier Summary:")
print(f"  • Total outliers detected: {total_outliers:,}")
print(f"  • Average outlier rate: {avg_outlier_pct:.2f}%")
print(f"  • Features analyzed: {len(outlier_analysis)}")

## 11. Train-Test Distribution Comparison

In [ ]:
# Compare distributions between train and test sets for key features
comparison_features = ['GrLivArea', 'YearBuilt', 'OverallQual', 'Neighborhood']
existing_comparison_features = [f for f in comparison_features if f in train_data.columns and f in test_data.columns]

if existing_comparison_features:
    print("🔍 TRAIN-TEST DISTRIBUTION COMPARISON")
    print("=" * 50)
    
    for feature in existing_comparison_features[:4]:  # Limit to 4 features
        print(f"\nComparing {feature} distribution...")
        compare_train_test_distributions(train_data, test_data, feature, figsize=(12, 6))
else:
    print("⚠️ No suitable features found for train-test comparison")

## 12. Comprehensive EDA Report Generation

In [ ]:
# Generate comprehensive EDA report
print("📋 GENERATING COMPREHENSIVE EDA REPORT...")
print("=" * 60)

eda_report = eda_analyzer.generate_eda_report()

# Print detailed summary
eda_analyzer.print_summary()

## 13. Key Insights and Recommendations

In [ ]:
# Compile key insights and recommendations
print("💡 KEY INSIGHTS AND RECOMMENDATIONS")
print("=" * 60)

# Data Quality Insights
print("\n🔍 DATA QUALITY INSIGHTS:")
if 'overall_quality' in quality_report:
    quality_score = quality_report['overall_quality']
    print(f"  • Overall Quality Score: {quality_score['overall_score']:.1f}/100 (Grade: {quality_score['grade']})")
    print(f"  • Assessment: {quality_score['interpretation']}")

missing_recs = quality_report['missing_values']['recommendations']
if missing_recs:
    print("\n📊 MISSING VALUES RECOMMENDATIONS:")
    for i, rec in enumerate(missing_recs, 1):
        print(f"  {i}. {rec}")

# Feature Engineering Insights
print("\n🔧 FEATURE ENGINEERING OPPORTUNITIES:")

if 'SalePrice' in train_data.columns:
    top_corr = eda_analyzer.target_correlation(5)
    print("  • Focus on top correlated features for modeling:")
    for _, row in top_corr.iterrows():
        print(f"    - {row['feature']} (correlation: {row['correlation']:.3f})")

if high_skew_features:
    print("  • Consider log transformation for highly skewed features:")
    for feature, skewness in high_skew_features[:5]:
        print(f"    - {feature} (skewness: {skewness:.2f})")

if high_zero_features:
    print("  • Consider binary encoding for features with many zeros:")
    for feature, count, percentage in high_zero_features[:3]:
        print(f"    - {feature} ({percentage:.1f}% zeros)")

# Modeling Recommendations
print("\n🤖 MODELING RECOMMENDATIONS:")
print("  • Apply robust scaling for features with outliers")
print("  • Consider ensemble methods to handle feature complexity")
print("  • Use cross-validation to assess model stability")
print("  • Monitor for overfitting given feature diversity")

if 'SalePrice' in train_data.columns:
    target_stats = eda_analyzer.analyze_target_variable()
    if target_stats['skewness'] > 1:
        print("  • Consider log transformation of target variable (SalePrice)")

print("\n✅ EDA ANALYSIS COMPLETED SUCCESSFULLY!")
print("\n🚀 Ready to proceed to Phase 3: Data Processing & Feature Engineering")

## 14. Export Results

In [ ]:
# Save key results for use in subsequent phases
import json
from datetime import datetime

# Prepare summary for export
eda_summary = {
    'timestamp': datetime.now().isoformat(),
    'dataset_info': {
        'train_shape': train_data.shape,
        'test_shape': test_data.shape,
        'numerical_features': len(train_data.select_dtypes(include=[np.number]).columns),
        'categorical_features': len(train_data.select_dtypes(include=['object']).columns)
    },
    'data_quality_grade': quality_report.get('overall_quality', {}).get('grade', 'N/A'),
    'missing_values_summary': quality_report['missing_values']['summary'],
    'recommendations': {
        'missing_values': missing_recs,
        'high_skew_features': [f[0] for f in high_skew_features[:10]] if 'high_skew_features' in locals() else [],
        'high_zero_features': [f[0] for f in high_zero_features[:5]] if 'high_zero_features' in locals() else []
    }
}

if 'SalePrice' in train_data.columns:
    top_corr = eda_analyzer.target_correlation(10)
    eda_summary['top_correlated_features'] = {
        row['feature']: row['correlation'] for _, row in top_corr.iterrows()
    }

# Save to file
output_path = '../data/processed/eda_summary.json'
with open(output_path, 'w') as f:
    json.dump(eda_summary, f, indent=2, default=str)

print(f"\n💾 EDA summary saved to: {output_path}")
print("\n📋 Summary includes:")
print("  • Dataset information and statistics")
print("  • Data quality assessment grade")
print("  • Missing values analysis")
print("  • Feature correlation rankings")
print("  • Recommendations for preprocessing")

print("\n🎉 Phase 2: Data Acquisition & Exploration - COMPLETED!")